# Generalized FPI Data Processing Notebook

This notebook provides a set of functions to load, clean, filter, and save FPI data from `.hdf5` files. The main logic is encapsulated in the `process_fpi_data` function, allowing it to be easily used for different datasets (like Lowell, Urbana, etc.) by simply changing the input parameters.

## 1. Imports



In [1]:
import h5py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
from datetime import datetime, timezone
import glob
import os

## 2. Helper Functions

These are general-purpose functions used in the main processing script.

In [2]:
def convert_timestamps_to_utc(timestamps):
    """
    Convert an array of UNIX timestamps to UTC datetime objects.
    """
    return [datetime.utcfromtimestamp(ts).replace(tzinfo=timezone.utc) for ts in timestamps]

def remove_outliers_by_sigma(df, sigma=3, columns=None):
    """
    Removes rows from a DataFrame if a value is an outlier based on the sigma rule, computed per hourly bin.

    An outlier is a value outside the range [mean - sigma * std, mean + sigma * std] for each bin.
    
    Parameters:
    - df (pd.DataFrame): Input DataFrame with a 'timestamps' column.
    - sigma (float): Number of standard deviations for the outlier threshold.
    - columns (list): Column names to check for outliers. Checks all numeric columns if None.

    Returns:
    - pd.DataFrame: A new DataFrame with outlier rows removed.
    """
    df_out = df.copy()
    columns_to_check = columns if columns is not None else df_out.select_dtypes(include=np.number).columns.tolist()

    df_out['datetime'] = pd.to_datetime(df_out['timestamps'], unit='s')
    
    # Binning by hour of the day. You can modify this for other frequencies.
    # For example, use `df_out['datetime'].dt.date` for daily bins.
    df_out['bin'] = df_out['datetime'].dt.hour

    grouped = df_out.groupby('bin')
    filtered_groups = []
    
    for bin_id, group in grouped:
        if group.empty:
            continue
        
        group_filtered = group.copy()
        for col in columns_to_check:
            if col in group_filtered.columns:
                mean_val = group_filtered[col].mean()
                std_val = group_filtered[col].std()

                if pd.isna(std_val) or std_val == 0:
                    continue

                lower_bound = mean_val - sigma * std_val
                upper_bound = mean_val + sigma * std_val
                group_filtered = group_filtered[(group_filtered[col] >= lower_bound) & (group_filtered[col] <= upper_bound)]
        
        filtered_groups.append(group_filtered)
    
    result = pd.concat(filtered_groups) if filtered_groups else pd.DataFrame(columns=df.columns)
    result = result.drop(columns=['datetime', 'bin'], errors='ignore')
    return result

## 3. Main Data Processing Function

This function orchestrates the entire workflow: loading, filtering, visualizing, and saving.

In [ ]:
def process_fpi_data(directory_path, site_name, output_base_path, oh_lower = 0, oh_upper = 200, sigma=3):
    """
    Loads, processes, and saves FPI data from a specified directory.

    Parameters:
    - directory_path (str): Path to the directory containing .hdf5 files.
    - site_name (str): Name of the site (e.g., 'Lowell') for titles and filenames.
    - output_base_path (str): Base directory to save processed data.
    - oh_lower (float): Lower bound for the relative emission brightness filter.
    - oh_upper (float): Upper bound for the relative emission brightness filter.
    - sigma (float): Sigma value for outlier clipping.
    """
    # --- 1. Load Data ---
    file_list = glob.glob(os.path.join(directory_path, '*.hdf5'))
    if not file_list:
        print(f"Error: No .hdf5 files found in '{directory_path}'")
        return
    
    print(f"Found {len(file_list)} files to process for {site_name}.")
    list_of_dfs = []
    required_keys = ['timestamps', 'cloudind', 'temp_err', 'wind_err', 'tn', 'dtn', 'vnu', 'dvnu', 'rlel', 'azm', 'elm']

    for file_path in file_list:
        try:
            with h5py.File(file_path, 'r') as f:
                if all(key in f for key in required_keys):
                    list_of_dfs.append(pd.DataFrame({key: f[key][:] for key in required_keys}))
                else:
                    print(f"Skipping file {os.path.basename(file_path)}: Missing required datasets.")
        except Exception as e:
            print(f"Could not process file {os.path.basename(file_path)}: {e}")

    if not list_of_dfs:
        print("No valid data loaded.")
        return

    df = pd.concat(list_of_dfs, ignore_index=True)
    df['rle_linear'] = 10**df['rlel']  # Convert log brightness to linear
    print(f"\nSuccessfully loaded data from {len(list_of_dfs)} files. Total rows: {len(df)}")
    print("-" * 50)

    # --- 2. Brightness Threshold Analysis ---
    print("Analyzing zenith data to help determine OH brightness threshold...")
    zenith_df = df[df['elm'] > 85].copy()

    if not zenith_df.empty:
        max_rle = zenith_df['rle_linear'].max()
        bins = np.arange(0, max_rle + 0.1, 0.1) # change the bin sizes as needed np.arange(0, max_rle + 0.5, 0.5) for example
        zenith_df['brightness_bin'] = pd.cut(zenith_df['rle_linear'], bins=bins, right=False)
        wind_deviation = zenith_df.groupby('brightness_bin')['vnu'].apply(lambda x: np.abs(x).mean()).reset_index()
        wind_deviation['bin_mid'] = wind_deviation['brightness_bin'].apply(lambda b: b.mid)

        plt.style.use('seaborn-v0_8-whitegrid')
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.plot(wind_deviation['bin_mid'], wind_deviation['vnu'], marker='o', linestyle='-', label='Mean Wind Deviation')
        ax.axhline(20, color='crimson', linestyle='--', label='20 m/s Threshold')
        ax.set_title(f'Wind Deviation vs. Brightness ({site_name} Zenith Data)')
        ax.set_xlabel('Linear Relative Emission (Brightness)')
        ax.set_ylabel('Mean Absolute Wind Speed |vnu| (m/s)')
        #ax.set_xlim(left=0, right=50)
        #ax.set_ylim(bottom=0)
        ax.legend()
        plt.show()
    else:
        print("No zenith data found; skipping OH threshold plot.")

    # --- 3. Quality Control Filtering ---
    print("\nApplying quality control filters...")
    original_rows = len(df)
    mask = (
        (df['cloudind'] < -25) &
        (df['temp_err'].isin([0, 1])) &
        (df['wind_err'].isin([0, 1])) &
        (df['dvnu'] < 100) &
        (df['tn'] > 150) &
        (df['dtn'] < 100) &
        (df['rle_linear'] >= oh_lower) & (df['rle_linear'] <= oh_upper)
    )
    clean_df = df[mask].copy()
    print(f"\nFiltering complete. Kept {len(clean_df)} of {original_rows} samples.")
    print("-" * 50)

    # --- 4. Sigma Outlier Clipping ---
    print(f"Applying {sigma}-sigma outlier clipping...")
    clipped_df = remove_outliers_by_sigma(clean_df, columns=['tn', 'vnu'], sigma=sigma)
    
    print(f"\nClipping complete. Original: {len(clean_df)}, After clipping: {len(clipped_df)}, Removed: {len(clean_df) - len(clipped_df)}")
    
    # --- 5. Visualization of Final Data ---
    fig, axs = plt.subplots(1, 2, figsize=(16, 5))
    axs[0].hist(clipped_df['rle_linear'], bins=50, color='royalblue', edgecolor='black')
    axs[0].set_title(f'Distribution of rle_linear in Final Data ({site_name})')
    axs[0].set_xlabel('Relative Emission (rle_linear)')
    axs[0].set_ylabel('Count')

    axs[1].scatter(clipped_df['rle_linear'], clipped_df['vnu'], alpha=0.3, color='darkorange', s=5)
    axs[1].set_title(f'Wind Velocity vs. rle_linear ({site_name})')
    axs[1].set_xlabel('Relative Emission (rle_linear)')
    axs[1].set_ylabel('Wind Velocity (vnu) (m/s)')
    plt.tight_layout()
    plt.show()

    # --- 6. Save Data ---
    output_directory = os.path.join(output_base_path, site_name)
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    output_file = os.path.join(output_directory, f'{site_name}_processed.nc')
    ds = clipped_df.to_xarray()
    ds.to_netcdf(output_file)
    print(f"\nFinal processed data saved to: {output_file}")

## 4. Example Usage

To run the processing for a new site, modify the variables in the cell below and run it. 

1.  **`SITE_DATA_DIR`**: Set this to the folder containing your `.hdf5` files.
2.  **`SITE_NAME`**: A descriptive name for your location (e.g., 'Urbana').
3.  **`OUTPUT_DIR`**: The base folder where the processed data will be saved.
4.  **`OH_LOWER`/`OH_UPPER`**: Adjust these based on the 'Wind Deviation vs. Brightness' plot generated for your data.

In [ ]:
if __name__ == "__main__":
    # --- Configuration ---
    SITE_DATA_DIR = '/mnt/madrigal_downloads/Lowell_FPI/' 
    SITE_NAME = 'Lowell'
    OUTPUT_DIR = '/mnt/madrigal_downloads/processedData/'
    
    # Brightness thresholds - adjust after reviewing the plot
    OH_LOWER_BOUND = 0
    OH_UPPER_BOUND = 200

    # Sigma for outlier clipping
    SIGMA_VALUE = 3
    
    # --- Run Processing ---
    process_fpi_data(
        directory_path=SITE_DATA_DIR,
        site_name=SITE_NAME,
        output_base_path=OUTPUT_DIR,
        oh_lower=OH_LOWER_BOUND,
        oh_upper=OH_UPPER_BOUND,
        sigma=SIGMA_VALUE
    )